<a href="https://colab.research.google.com/github/mch663/Sparse-Zoo/blob/main/Q.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#配置Qserve以及环境搭建

In [3]:
!conda create -n QServe python=3.10 -y
!conda activate QServe

/bin/bash: line 1: conda: command not found
/bin/bash: line 1: conda: command not found


In [4]:
!git clone https://github.com/mit-han-lab/qserve
%cd qserve

Cloning into 'qserve'...
remote: Enumerating objects: 312, done.
remote: Counting objects: 100% (312/312), done.
remote: Compressing objects: 100% (201/201), done.
remote: Total 312 (delta 138), reused 259 (delta 106), pack-reused 0 (from 0)
Receiving objects: 100% (312/312), 3.39 MiB | 27.73 MiB/s, done.
Resolving deltas: 100% (138/138), done.
/content/qserve


In [5]:
!pip install --upgrade pip  # 启用 PEP 660 支持
# 如果你希望使用内置的 nvcc
!conda install -c nvidia cuda-toolkit -y
!pip install -e .  # 安装 QServe
!pip install flash-attn --no-build-isolation  # 安装 FlashAttention-2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 58.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
/bin/bash: line 1: conda: command not found
Obtaining file:///content/qserve
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of mdit-py-plugins to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of mdit-py-plugins to determine which version is compatible with 

In [6]:
%cd kernels
!python setup.py install
%cd ..

流式输出内容被截断，只能显示最后 5000 行内容。
            instantiation of "void invokeApplyBiasRopeUpdateKVCache<T,KVCacheBuffer,IS_GENERATE>(T *, T *, KVCacheBuffer &, const T *, const int *, const int *, const int *, int, int, int, int, int, int, int, int, int, float, RotaryScalingType, float, int, PositionEmbeddingType, const int *, __nv_bool, const float *, int, KvCacheDataType, __nv_bool, int, int2 &, cudaStream_t) [with T=half, KVCacheBuffer=KVLinearBuffer, IS_GENERATE=true]" at line 17 of csrc/fused_attention/update_kv_cache.cu

csrc/fused_attention/applyBiasRopeUpdateKVCache.h(416): warning #549-D: variable "cur_v_zeros" is used before its value is set
                              mmha::store_8bits_kv_cache_vec(vDst, v, inBlockIdx, cur_v_scale_orig_quant, cur_v_zeros);
                                                                                                          ^
          detected during:
            instantiation of "void applyBiasRopeUpdateKVCache<T,T_cache,Dh_MAX,ADD_BIAS,STORE_

In [ ]:
# 切换到根目录
%cd /

# 克隆 VILA 仓库
!git clone https://github.com/NVlabs/VILA

# 进入 VILA 目录
%cd VILA

# 安装 flash-attention
!pip install https://github.com/Dao-AILab/flash-attention/releases/download/v2.5.8/flash_attn-2.5.8+cu122torch2.3cxx11abiFALSE-cp310-cp310-linux_x86_64.whl

# 安装 VILA
!pip install -e .
!pip install -e ".[train]"

/
fatal: destination path 'VILA' already exists and is not an empty directory.
/VILA
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.9/120.9 MB 49.8 MB/s eta 0:00:00
Obtaining file:///VILA
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Cloning https://github.com/bfshi/scaling_on_scales to /tmp/pip-install-flp03gzu/s2wrapper_e2c81ff0115244cdbc0d0b487a3914d8
  Running command git clone --filter=blob:none --quiet https://github.com/bfshi/scaling_on_scales /tmp/pip-install-flp03gzu/s2wrapper_e2c81ff0115244cdbc0d0b487a3914d8
  Resolved https://github.com/bfshi/scaling_on_scales to commit 60da2afe5121533947c1dff2d57d9c583516a157
  Preparing metadata (setup.py) ... done
  Building editable for vila (pyproject.toml) ... done
  Created wheel for vila: filename=vila-2.0.0-0.editable-py3-none-any.whl size=15033 sha256=9aeedddc525201f0

In [7]:
!git lfs install

Updated git hooks.
Git LFS initialized.


In [8]:
!mkdir -p qserve_checkpoints
%cd qserve_checkpoints

/content/qserve/qserve_checkpoints


In [9]:
!git clone https://huggingface.co/mit-han-lab/Llama-3-8B-QServe

Cloning into 'Llama-3-8B-QServe'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 13 (delta 0), reused 0 (delta 0), pack-reused 4 (from 1)
Unpacking objects: 100% (13/13), 2.23 MiB | 6.52 MiB/s, done.
You can inspect what was checked out with 'git status'
and retry with 'git restore --source=HEAD :/'


Exiting because of "interrupt" signal.
^C


# 设置环境变量并运行基准测试：

In [ ]:
import os
os.environ['MODEL_PATH'] = './qserve_checkpoints/Llama-3-8B-QServe'
os.environ['GLOBAL_BATCH_SIZE'] = '128'

In [ ]:
# 设置模型路径
model_path = './qserve_checkpoints/Llama-3-8B-QServe'